In this notebook I demonstrate my pipeline for training a machine learning model for algo trading. 

This notebook walks you through:
* loading bars data
* extracting features based on technical indicators
* labelling target variables for modelling using the triple barrier method
* dynamically adjusting stop loss and take profit levels
* selecting useful features using recursive feature elimination with random forests OR based on your own reasoning
* training an XGBoost model to make buy decisions
* backtesting and evaluating model performance

I am using a similar approach to iterate over different strategies and trade stocks using the Alpaca API. If you would like to know more or see a more detailed implementation please contact me:


In [81]:
import pandas as pd
from stonks import feature_extraction, feature_selection, backtest, labelling, models, visualisation
# from datetime import datetime, timedelta
# from pytz import timezone

In [82]:
# Loading stock price data.
# I get minute bars data from the Alpaca API. For your convenience, I am going to load data from a csv file. 
# This gives you 1 minute bars for PLTR (Palantir) from 2022-05-01.

hist = pd.read_csv('data/bars_PLTR.csv', index_col='timestamp')
hist.index = pd.DatetimeIndex(hist.index)
hist.head()

,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2022-05-02 09:30:00+00:00,10.380,10.3900,10.2800,10.3300,583607,1561,10.333511
2022-05-02 09:31:00+00:00,10.325,10.3263,10.2600,10.2699,201313,731,10.280256
2022-05-02 09:32:00+00:00,10.260,10.2950,10.2175,10.2900,271984,1168,10.254898
2022-05-02 09:33:00+00:00,10.280,10.2900,10.2200,10.2300,152645,685,10.259573
2022-05-02 09:34:00+00:00,10.230,10.2850,10.1918,10.2800,311727,1016,10.237203


In [83]:
# If you would like to get your own minute bars, you can create a Free Alpaca account and use this code:
# from alpaca_trade_api.rest import REST
# from alpaca.data.timeframe import TimeFrame, TimeFrameUnit

# API_KEY = "YOURKEY"
# SECRET_KEY = "YOURKEY"
# api = REST(key_id=API_KEY,secret_key=SECRET_KEY,base_url="https://paper-api.alpaca.markets")
# symbol = ['PLTR']
# hist=api.get_bars(symbol,timeframe=TimeFrame(amount=1, unit=TimeFrameUnit.Minute), start='2022-05-01').df
# hist = hist.drop(columns = ['symbol'])

In [84]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

In [85]:
# Now that we have bars data we can extract features that we will use to train our machine learning model
# The features consist of various technical indicators such as moving averages, volatility indicators (rolling standard error, ATR), oversold/ overbought indicators etc.
# I create features algorithmically by iterating over different time windows e.g. macd_12_26 compares exponential moving averages over the last 12 and 26 minutes, but it could be 
# [26,50], [100,1000], [100,9000] minutes.

# Turn off warnings so pandas does not print a long list of them for each column added with .loc
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
#SettingWithCopyWarning

# lets limit the size of windows we consider to 300 minutes, to speed things up
hist = feature_extraction.get_features(hist, window_limit=9000)

meow


In [86]:
# the features are added to the dataframe and given a descriptive feature name
# for example macd_12_26 - is the moving average convergence divergence for the 12 and 26 minute windows
# vol_std_50 - is volatility measured as a rolling exponentially averaged standard deviation over 50 minutes
hist.tail()

,open,high,low,close,volume,trade_count,vwap,returns,volume_pct_change,minutes_open,date,datetime,market_open,close_market_open,datetime_market_open,pct_change_daily,30_min_bin,first_30_min,last_30_min,ATR_1,RSI_1,ema_1,ema_volume_1,vol_std_1,vol_std_volume_1,vol_pct_1,vol_pct_volume_1,returns_vol_std_1,swing_low_1,swing_high_1,swing_low_pct_1,swing_high_pct_1,min_vol_std_1,max_vol_std_1,close_ema_vol_std_1,close_ema_pct_1,ema_vol_std_shift_1,vol_std_pct_1,vol_std_pct_ratio_1,returns_ema_1,exp_ema_pct_shift_1,exp_ema_pct_abs_shift_1,exp_ema_pct_dir_shift_1,exp_pct_volatility_1,exp_pct_volatility_old_1,ATR_2,RSI_2,ADX_2,ema_2,ema_volume_2,vol_std_2,vol_std_volume_2,vol_pct_2,vol_pct_volume_2,returns_vol_std_2,swing_low_2,swing_high_2,swing_low_pct_2,swing_high_pct_2,min_vol_std_2,max_vol_std_2,close_ema_vol_std_2,close_ema_pct_2,ema_vol_std_shift_2,vol_std_pct_2,vol_std_pct_ratio_2,returns_ema_2,exp_ema_pct_shift_2,exp_ema_pct_abs_shift_2,exp_ema_pct_dir_shift_2,exp_pct_volatility_2,exp_pct_volatility_old_2,ATR_3,RSI_3,ADX_3,ema_3,ema_volume_3,vol_std_3,vol_std_volume_3,vol_pct_3,vol_pct_volume_3,returns_vol_std_3,swing_low_3,swing_high_3,swing_low_pct_3,swing_high_pct_3,min_vol_std_3,max_vol_std_3,close_ema_vol_std_3,close_ema_pct_3,ema_vol_std_shift_3,vol_std_pct_3,vol_std_pct_ratio_3,returns_ema_3,exp_ema_pct_shift_3,exp_ema_pct_abs_shift_3,exp_ema_pct_dir_shift_3,exp_pct_volatility_3,exp_pct_volatility_old_3,ATR_5,RSI_5,CCI_5,ADX_5,ema_5,ema_volume_5,vol_std_5,vol_std_volume_5,vol_pct_5,vol_pct_volume_5,returns_vol_std_5,swing_low_5,swing_high_5,swing_low_pct_5,swing_high_pct_5,min_vol_std_5,max_vol_std_5,close_ema_vol_std_5,close_ema_pct_5,ema_vol_std_shift_5,vol_std_pct_5,vol_std_pct_ratio_5,returns_ema_5,exp_ema_pct_shift_5,exp_ema_pct_abs_shift_5,exp_ema_pct_dir_shift_5,exp_pct_volatility_5,exp_pct_volatility_old_5,ATR_8,RSI_8,CCI_8,ADX_8,ema_8,ema_volume_8,vol_std_8,vol_std_volume_8,vol_pct_8,vol_pct_volume_8,returns_vol_std_8,swing_low_8,swing_high_8,swing_low_pct_8,swing_high_pct_8,min_vol_std_8,max_vol_std_8,close_ema_vol_std_8,close_ema_pct_8,ema_vol_std_shift_8,vol_std_pct_8,vol_std_pct_ratio_8,returns_ema_8,exp_ema_pct_shift_8,exp_ema_pct_abs_shift_8,exp_ema_pct_dir_shift_8,exp_pct_volatility_8,exp_pct_volatility_old_8,ATR_12,RSI_12,CCI_12,ADX_12,ema_12,ema_volume_12,vol_std_12,vol_std_volume_12,vol_pct_12,vol_pct_volume_12,returns_vol_std_12,swing_low_12,swing_high_12,swing_low_pct_12,swing_high_pct_12,min_vol_std_12,max_vol_std_12,close_ema_vol_std_12,close_ema_pct_12,ema_vol_std_shift_12,vol_std_pct_12,vol_std_pct_ratio_12,returns_ema_12,exp_ema_pct_shift_12,exp_ema_pct_abs_shift_12,exp_ema_pct_dir_shift_12,exp_pct_volatility_12,exp_pct_volatility_old_12,ATR_20,RSI_20,CCI_20,ADX_20,ema_20,ema_volume_20,vol_std_20,vol_std_volume_20,vol_pct_20,vol_pct_volume_20,returns_vol_std_20,swing_low_20,swing_high_20,swing_low_pct_20,swing_high_pct_20,min_vol_std_20,max_vol_std_20,close_ema_vol_std_20,close_ema_pct_20,ema_vol_std_shift_20,vol_std_pct_20,vol_std_pct_ratio_20,returns_ema_20,exp_ema_pct_shift_20,exp_ema_pct_abs_shift_20,exp_ema_pct_dir_shift_20,exp_pct_volatility_20,exp_pct_volatility_old_20,ATR_26,RSI_26,CCI_26,ADX_26,ema_26,ema_volume_26,vol_std_26,vol_std_volume_26,vol_pct_26,vol_pct_volume_26,returns_vol_std_26,swing_low_26,swing_high_26,swing_low_pct_26,swing_high_pct_26,min_vol_std_26,max_vol_std_26,close_ema_vol_std_26,close_ema_pct_26,ema_vol_std_shift_26,vol_std_pct_26,vol_std_pct_ratio_26,returns_ema_26,exp_ema_pct_shift_26,exp_ema_pct_abs_shift_26,exp_ema_pct_dir_shift_26,exp_pct_volatility_26,exp_pct_volatility_old_26,ATR_40,RSI_40,CCI_40,ADX_40,ema_40,ema_volume_40,vol_std_40,vol_std_volume_40,vol_pct_40,vol_pct_volume_40,returns_vol_std_40,swing_low_40,swing_high_40,swing_low_pct_40,swing_high_pct_40,min_vol_std_40,max_vol_std_40,close_ema_vol_std_40,close_ema_pct_40,ema_vol_std_shift_40,vol_std_pct_40,vol_std_pct_ratio_40,returns_ema_40,exp_ema_pct_shift_40,exp_ema_pct_abs_shift_40,exp_ema_pct_

In [87]:
# even with this limit we have quite a lot of features to consider
hist.shape

(134053, 1182)

In [88]:
# Now, in order to train the model we need to label desired outcomes (targets).

# So far the best method I have found is the triple barrier method (see Lopez De Prado)
# The triple barrier considers future stock price over a specified time window, e.g. 50 minutes. 
# If the price hits the upper barrier first (increases by X amount) it gets a 1, if it hits the lower barrier first it gets a -1. 
# If it doesnt cross either barrier it gets a 0
# The barriers could be set to a fixed number, such as 1% or 1 dollar above current price
# However, I will set them dynamically by considering the recent trend and volatility

# This multiplier gives us the expected volatility over a 50 minute time period expressed as pct. 
# I calculate the standard deviation over the last 50 min expressed as % of the 50 minute EMA.
# then I take the exponential moving average of the last 5000 rows of volatilities
hist['multiplier_volatility'] = hist['exp_pct_volatility_50']
print('Expected volatility over a 50 minute time period (% change as decimal points):')
print(hist['multiplier_volatility'].describe())
print('')


# In order to take a reasonable take profit level we can also add the trend to volatility - by how many pct, does the ema typically increase over a 50m window?
hist['multiplier'] = hist['exp_ema_pct_abs_shift_50']
print('Expected EMA shift over a 50 minute time period (% change as decimal points):')
print(hist['multiplier'].describe())

Expected volatility over a 50 minute time period (% change as decimal points):
count    134004.000000
mean          0.009144
std           0.002030
min           0.005583
25%           0.008001
50%           0.008723
75%           0.009877
max           0.020173
Name: multiplier_volatility, dtype: float64

Expected EMA shift over a 50 minute time period (% change as decimal points):
count    133954.000000
mean          0.007825
std           0.001627
min           0.003336
25%           0.006859
50%           0.007548
75%           0.008510
max           0.016046
Name: multiplier, dtype: float64


In [89]:
# then we obtain our barriers by adding the multipliers to the current price. I have found that the volatility multiplier is sufficient.
hist['higher_bound'] = hist.close * (1 + (hist.multiplier * 0 + hist.multiplier_volatility * 1))
hist['lower_bound'] = hist.close * (1 - (hist.multiplier * 0 + hist.multiplier_volatility * 1))

In [90]:
# lets use the triple barrier function to label every minute bar based on whether price will hit our desired target in the next 50 minutes
window = 26
hist = labelling.get_barrier(hist,hist.close.values,hist.higher_bound.values,hist.lower_bound.values,window=window,side=None)
hist['barrier'].value_counts()

 0.0    86013
-1.0    24022
 1.0    23993
Name: barrier, dtype: int64

In [91]:
# some algorithms like Keras LSTMs like this labelling more
hist['barrier'].loc[hist['barrier'] == 1] = 2
hist['barrier'].loc[hist['barrier'] == 0] = 1
hist['barrier'].loc[hist['barrier'] == -1] = 0

/var/folders/fx/9gkklqr57dgg7dv2kk03gpkw0000gn/T/ipykernel_39767/2006840587.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hist['barrier'].loc[hist['barrier'] == 1] = 2
/var/folders/fx/9gkklqr57dgg7dv2kk03gpkw0000gn/T/ipykernel_39767/2006840587.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hist['barrier'].loc[hist['barrier'] == 0] = 1
/var/folders/fx/9gkklqr57dgg7dv2kk03gpkw0000gn/T/ipykernel_39767/2006840587.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

In [92]:
# As an option, we can eliminate the last n minutes of each day so that the algorithm does not get confused trying to predict next day morning prices
# window = 26
# hist = hist.loc[hist.minutes_open < (hist.minutes_open.max() - window)]


In [93]:
# Now that we have created our features and target variables, we have to reduce thenumber of features we are considering to avoid overfitting and interpet the model
# We can use different methods for it like recursive feature elemination (RFE), genetic algorithms or mutual information.
# here is an example using RFE.

# Running the below cell takes 18m on my machine
# If you want to skip this just use the model features I get as a result of my run (provided)

In [ ]:
feature_ranking = feature_selection.select_best_features(hist.drop(columns=['date','datetime','market_open','datetime_market_open']), 
                                       method='RFE', 
                                       estimator_type = 'random_forest', 
                                       #class_weight = {0:5,1:1,2:10},
                                       max_depth = 4, 
                                       n_features_to_select=10,
                                       step = 20)

f = pd.Series(feature_ranking,index=hist.drop(columns=['barrier','date','datetime','market_open','datetime_market_open']).columns)
f.loc[f==1].index

In [ ]:
pd.set_option('display.max_rows',1000)

In [100]:
# here are the features I get as a result of RFE:
model_features = ['minutes_open', '30_min_bin', 'returns_vol_std_20',
       'volume_macd_50_100', 'ema_volume_vol_std_50_200',
       'volume_macd_pct_50_200', 'volume_macd_60_150',
       'ema_volume_vol_std_60_150', 'ema_volume_vol_std_new_60_150',
       'volume_macd_pct_60_150']

In [96]:
# RFE with 20 features step 10
model_features = ['minutes_open', '30_min_bin', 'last_30_min', 'returns_vol_std_12',
       'returns_vol_std_20', 'returns_vol_std_26', 'returns_vol_std_40',
       'returns_vol_std_50', 'exp_pct_volatility_150', 'volume_macd_50_100',
       'ema_volume_vol_std_50_100', 'volume_macd_pct_50_100',
       'volume_macd_50_200', 'ema_volume_vol_std_50_200',
       'ema_volume_vol_std_new_50_200', 'volume_macd_pct_50_200',
       'volume_macd_60_150', 'ema_volume_vol_std_60_150',
       'ema_volume_vol_std_new_60_150', 'volume_macd_pct_60_150']

In [107]:
# hand picked features by logic
model_features = ['swing_high_pct_12', 'volume_macd_pct_12_100',
       'swing_low_pct_50', 'macd_pct_26_100',
       'swing_low_pct_100', 'swing_high_pct_100', 'vol_std_pct_100',
       'exp_ema_pct_abs_shift_50', 'swing_high_pct_26',
       'exp_pct_volatility_50', 'volume_macd_pct_26_100', '30_min_bin',
       'vol_std_pct_26', 'minutes_open']

In [108]:
# add barrier to the list of features as the next function will use it
model_features = model_features + ['barrier']

In [102]:
# Now it is time to train our model
# I am using gradient boosting to predict which barrier the stock price will hit first. 
# The backtest days parameter controls what time period to set aside for backtesting. Lets set aside the last 30 days for testing. 

# Be mindful that the classification metrics are hard to interpret. 
# I usually get a low f-1 score for the target class (2) ~30, but the winrate may come out to 50% with a 2:1 win to loss ratio and decent returns.

ypred, model_index,model = models.predict_barrier(hist, 
                                                  features = model_features, 
                                                  max_depth=4, 
                                                  model_type='boosting', 
                                                  parallel=True, 
                                                  normalise=False,
                                                  tree_method='hist',
                                                  backtest_days=30,
                                                  prod_model=False)

              precision    recall  f1-score   support

         0.0       0.42      0.22      0.29      1266
         1.0       0.83      0.96      0.89      5901
         2.0       0.44      0.33      0.38      1410

    accuracy                           0.75      8577
   macro avg       0.56      0.50      0.52      8577
weighted avg       0.70      0.75      0.72      8577



In [103]:
# which features are most useful for our predictions?
# for my model, the most useful features turned out to be minutes_open (how long the market has been open)
# and the volume_macd_pct_60_150 - the MACD for 60 and 150 minutes expressed as pct.
# this example also shows that relying solely on feature selection algorithms is not a good idea 
# as it identified three features with highly overlapping information
# volume_macd_pct_60_150, ema_volume_vol_std_60_150 and volume_macd_60_150 are the same feature with different transformations, so its best to pick one
pd.set_option('display.max_rows',1000)
pd.Series(model.feature_importances_,index=model.feature_names_in_).sort_values()

ema_volume_vol_std_60_150        0.036540
ema_volume_vol_std_new_60_150    0.047213
ema_volume_vol_std_50_200        0.066125
30_min_bin                       0.068681
volume_macd_pct_50_200           0.069833
volume_macd_pct_60_150           0.090011
returns_vol_std_20               0.109757
volume_macd_50_100               0.130391
minutes_open                     0.185698
volume_macd_60_150               0.195751
dtype: float32

In [104]:
# use the model output to create a dataframe for backtesting 
backtest_hist = hist.loc[model_index]
backtest_hist['ml_prediction'] = ypred
backtest_hist['strategy1_sell'] = 0
backtest_hist['strategy1_buy'] = 0

# whenever the model predicts that we will hit the upper barrier, we will take it as a buy signal
backtest_hist['strategy1_buy'].loc[(backtest_hist.ml_prediction == 2)] = 1

# optionally, we can use the same model to generate sell signals
#backtest_hist['strategy1_sell'].loc[(backtest_hist.ml_prediction == 0)] = 1

# how many buy signals (1) do we get in our sample?
backtest_hist.strategy1_buy.value_counts()

/var/folders/fx/9gkklqr57dgg7dv2kk03gpkw0000gn/T/ipykernel_39767/4066975071.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  backtest_hist['strategy1_buy'].loc[(backtest_hist.ml_prediction == 2)] = 1


0    7516
1    1061
Name: strategy1_buy, dtype: int64

In [105]:
# before we backtest, we need to set the stop loss and take profit levels
# these can be different from the barriers we used during labelling
# lets use 26 minute volatility and a 3:1 ratio of take profit to stop loss

backtest_hist['multiplier'] = backtest_hist['exp_ema_pct_abs_shift_26']
backtest_hist['multiplier_volatility'] = backtest_hist['exp_pct_volatility_26']

backtest_hist['higher_bound'] = backtest_hist.close * (1 + ((0 * backtest_hist.multiplier + 3 * backtest_hist.multiplier_volatility) * 1))
backtest_hist['lower_bound'] = backtest_hist.close * (1 - ((0 * backtest_hist.multiplier + 1 * backtest_hist.multiplier_volatility) * 1))

In [106]:
# the backtest iterates over our test dataframe and simulates trading decisions
# a buy is made based on the ML model predictions
# a sell decision can be made based on the ML model and/ or crossing stop loss take profit (SLTP) levels.
# these backtest params use the ML model to make buy decisions and SLTP to sell. 
# the holding period is 60, meaning that if no barrier is hit in 60 minutes we will sell.
# in addition, we will hold stock if there is a new buy signal, even if we are below/ above SLTP level.

# with the automated features the model is not performing well. the wins are bigger than losses, but not enough
trades = backtest.run_backtest(
    backtest_hist=backtest_hist.loc[backtest_hist.lower_bound.notna()], 
    buy_index=backtest_hist.loc[backtest_hist.strategy1_buy==1].index, 
    sell_index=[],
    take_profit_series=backtest_hist.higher_bound,
    stop_loss_series=backtest_hist.lower_bound,
    risk=1,
    cash = 10000,
    holding_period= 30,
    min_holding_period=0,
    hold_on_buy_signal=False,
    sltp=True,
    sltp_update=False)

Cash money:
9793.476002842042
112
112
Winrate:
0.2857142857142857
Avg win:
0.012199857873811865
Avg loss:
-0.006830136406268766


In [109]:
ypred, model_index,model = models.predict_barrier(hist, 
                                                  features = model_features, 
                                                  max_depth=4, 
                                                  model_type='boosting', 
                                                  parallel=True, 
                                                  normalise=False,
                                                  tree_method='hist',
                                                  backtest_days=30,
                                                  prod_model=False)

              precision    recall  f1-score   support

         0.0       0.42      0.31      0.36      1266
         1.0       0.83      0.95      0.89      5901
         2.0       0.48      0.33      0.39      1410

    accuracy                           0.75      8577
   macro avg       0.58      0.53      0.54      8577
weighted avg       0.71      0.75      0.73      8577



In [110]:
# use the model output to create a dataframe for backtesting 
backtest_hist = hist.loc[model_index]
backtest_hist['ml_prediction'] = ypred
backtest_hist['strategy1_sell'] = 0
backtest_hist['strategy1_buy'] = 0

# whenever the model predicts that we will hit the upper barrier, we will take it as a buy signal
backtest_hist['strategy1_buy'].loc[(backtest_hist.ml_prediction == 2)] = 1

# optionally, we can use the same model to generate sell signals
#backtest_hist['strategy1_sell'].loc[(backtest_hist.ml_prediction == 0)] = 1

# how many buy signals (1) do we get in our sample?
backtest_hist.strategy1_buy.value_counts()

/var/folders/fx/9gkklqr57dgg7dv2kk03gpkw0000gn/T/ipykernel_39767/4066975071.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  backtest_hist['strategy1_buy'].loc[(backtest_hist.ml_prediction == 2)] = 1


0    7620
1     957
Name: strategy1_buy, dtype: int64

In [111]:
backtest_hist['multiplier'] = backtest_hist['exp_ema_pct_abs_shift_26']
backtest_hist['multiplier_volatility'] = backtest_hist['exp_pct_volatility_26']

backtest_hist['higher_bound'] = backtest_hist.close * (1 + ((0 * backtest_hist.multiplier + 3 * backtest_hist.multiplier_volatility) * 1))
backtest_hist['lower_bound'] = backtest_hist.close * (1 - ((0 * backtest_hist.multiplier + 1 * backtest_hist.multiplier_volatility) * 1))

In [112]:
trades = backtest.run_backtest(
    backtest_hist=backtest_hist.loc[backtest_hist.lower_bound.notna()], 
    buy_index=backtest_hist.loc[backtest_hist.strategy1_buy==1].index, 
    sell_index=[],
    take_profit_series=backtest_hist.higher_bound,
    stop_loss_series=backtest_hist.lower_bound,
    risk=1,
    cash = 10000,
    holding_period= 30,
    min_holding_period=0,
    hold_on_buy_signal=False,
    sltp=True,
    sltp_update=False)

Cash money:
10700.544221768389
98
98
Winrate:
0.2755102040816326
Avg win:
0.015003647388568185
Avg loss:
-0.0066050339319929


In [ ]:
# the algorithm gives me 10700, so 7% return in a month, with a winrate of 28%, and the average win size of 1.5% and loss size of 0.66 (more than 2:1 ratio)

In [ ]:
# these strategies decay with time and based on changing market condition
# in order to make sure consistent returns we need to iterate over strategies using different parameters, features and stocks
# This is what I am working on now - contact me to find out more.

In [ ]:
# review simulated trades 
trades['date'] = trades.buy_index.apply(lambda x: x.date())
trades.tail(50)

In [ ]:
show_trades(hist,trades,day='2023-04-11')

In [ ]:
# now lets do the same but selecting features by hand.